# To DO

figure out how to add features to decoder

In [1]:
import numpy as np

In [2]:
#this is the encoder sim_c
#set up a state space model sort of thing
from riglib.bmi.state_space_models import StateSpaceEndptVel2D
ssm = StateSpaceEndptVel2D()

# specify a control matrix
N_NEURONS = 20
N_STATES = 7  # 3 positions and 3 velocities and an offset


# build the observation matrix
sim_C = np.zeros((N_NEURONS, N_STATES))
# control x positive directions
sim_C[0, :] = np.array([0, 0, 0, 10, 0, 0, 0])
sim_C[1, :] = np.array([0, 0, 0, -10, 0, 0, 0])
# control z positive directions
sim_C[2, :] = np.array([0, 0, 0, 0, 0, 10, 0])
sim_C[3, :] = np.array([0, 0, 0, 0, 0, -10, 0])

# Select features

## check that decoder works with the same number of encoder features

In [3]:
#  for initialization, we just train the decoder, right.
#initialize the goal estimator
#fb_ctrl = LQRController(A, B, Q, R)

#initialize the encoder
import riglib.bmi.sim_neurons as sim_neurons
np.random.seed(seed= 6)
encoder = sim_neurons.GenericCosEnc(sim_C, ssm,
                                           call_ds_rate=1) #for sim, we want this to return us every call, 

intended_state = np.asarray([0, 0, 0, 10, 0, 10, 1]).T #transpose into a col vector
neural_obs = encoder(intended_state)

print('neural obs')
print(neural_obs)
print()

#initialize the decoder
from features import simulation_features
decoder =  simulation_features.SimKFDecoderSup()
decoder.load_decoder(supplied_encoder=encoder, 
                    supplied_SSM=ssm)

print('decoder decoded state')
print(decoder.decoder(neural_obs))

neural obs
[15  0 13  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]



/home/sijia-aw/BMi3D_my/lab_bmi3d/riglib/blackrock/cerelink.py:13: UserWarning: Unable to import cerebus library. Check if is installed if using the Blackrock NeuroPort system
  warnings.warn("Unable to import cerebus library. Check if is installed if using the Blackrock NeuroPort system")


Creating simulation decoder..
SimKFDecoderSup:loaded suppled_encoder from function input
<riglib.bmi.sim_neurons.GenericCosEnc object at 0x7f8fdc92f790> <class 'riglib.bmi.sim_neurons.GenericCosEnc'>
SimKFDecoderSup:loaded suppled_ssm from function input
<riglib.bmi.sim_neurons.GenericCosEnc object at 0x7f8fdc92f790> <class 'riglib.bmi.sim_neurons.GenericCosEnc'>
SimKFDecoderSup: units:  20
 SimKFDecoderSup: zscore decoder ? :  False
{}
end of kwargs
('zscore value: ', False, <class 'bool'>)
no init_zscore
decoder decoded state
[[ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [11.21199619]
 [ 0.        ]
 [10.10656804]
 [ 1.        ]]


In [4]:
decoder.decoder.n_features

20

In [5]:
decoder.decoder.n_units

20

In [6]:
decoder.decoder.filt.A

matrix([[1. , 0. , 0. , 0.1, 0. , 0. , 0. ],
        [0. , 1. , 0. , 0. , 0.1, 0. , 0. ],
        [0. , 0. , 1. , 0. , 0. , 0.1, 0. ],
        [0. , 0. , 0. , 0.8, 0. , 0. , 0. ],
        [0. , 0. , 0. , 0. , 0.8, 0. , 0. ],
        [0. , 0. , 0. , 0. , 0. , 0.8, 0. ],
        [0. , 0. , 0. , 0. , 0. , 0. , 1. ]])

## here we implement the change

In [7]:
# here we implement the change 

# select features

In [8]:
selected_indices = [0,1,2,3]

trans_neural_obs =  neural_obs[selected_indices]

print(trans_neural_obs)

[15  0 13  0]


In [9]:
print(decoder.decoder(neural_obs))

[[ 1.1905265 ]
 [ 0.        ]
 [ 1.07370199]
 [13.36238072]
 [ 0.        ]
 [12.05816031]
 [ 1.        ]]


In [10]:
try:
    print(decoder.decoder(trans_neural_obs))
except:
    print('this did not work, as expected ')

this did not work, as expected 


In [11]:
from weights import change_target_kalman_filter_with_a_C_mat

def select_decoder_features(target_decoder, selected_indices, debug = True):
    '''
    
    '''
    #update the C, Q matrix
    prev_C = target_decoder.filt.C
    transformed_C = prev_C[selected_indices,:]
    
    prev_Q = target_decoder.filt.Q
    transformed_Q = prev_Q[selected_indices, :][:,selected_indices]
    
    #use the updated C,Q matrices to update the decoder
    change_target_kalman_filter_with_a_C_mat(target_decoder.filt, transformed_C, 
                                     Q = transformed_Q, 
                                     debug = False
                                    )
    
    #update the feature count
    target_decoder.n_features = target_decoder.filt.C.shape[0]
    #To-DO: also need to change how many feats we need
    

    


select_decoder_features(decoder.decoder, selected_indices)
print(decoder.decoder(trans_neural_obs))

[[ 2.58343409]
 [ 0.        ]
 [ 2.33106053]
 [13.71397982]
 [ 0.        ]
 [12.37914781]
 [ 1.        ]]


In [12]:
decoder.decoder.filt.Q.shape

(4, 4)

## add features

In [13]:
def add_new_features(target_decoder, num_add_feat):
    '''
    
    '''

    (num_old_feats,num_states) = target_decoder.filt.C.shape
    num_new_feats = num_add_feat + num_old_feats

    #add zero rows to the observation matrix
    prev_C = target_decoder.filt.C
    new_rows = np.zeros((num_add_feat, num_states))
    new_C =  np.vstack((prev_C, new_rows))

    new_C.shape

    #add rows and columns to the observation matrix
    prev_Q = target_decoder.filt.Q
    new_rows = np.zeros((num_add_feat, num_old_feats))
    new_Q = np.vstack((prev_Q, new_rows))

    new_cols = np.zeros((num_new_feats, num_add_feat))
    new_Q = np.hstack((new_Q, new_cols))

    # assign back to the decoder
    change_target_kalman_filter_with_a_C_mat(target_decoder.filt, new_C, Q = new_Q,
                                            debug = False)

    
add_new_features(decoder.decoder,  2)

C matrix before
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  4.96162714e-01
   0.00000000e+00  4.83642725e-04  1.04880308e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -4.94739263e-01
   0.00000000e+00 -3.62002311e-03  1.05552288e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.57372307e-02
   0.00000000e+00  5.05239899e-01  1.04127074e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -2.47342267e-02
   0.00000000e+00 -5.01282137e-01  1.03067398e+00]]
C matrix after
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  4.96162714e-01
   0.00000000e+00  4.83642725e-04  1.04880308e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -4.94739263e-01
   0.00000000e+00 -3.62002311e-03  1.05552288e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.57372307e-02
   0.00000000e+00  5.05239899e-01  1.04127074e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -2.47342267e-02
   0.00000000e+00 -5.01282137e-01  1.03067398e+00]
 [ 0.00000000e+00  0.00000000e+0

### save the C matrix

In [14]:
print(decoder.decoder.filt.C.shape)

(6, 7)


In [15]:
debug = True
prev_C = decoder.decoder.filt.C
transformed_C = prev_C[selected_indices,:]

prev_Q = decoder.decoder.filt.Q

transformed_Q = prev_Q[selected_indices, :][:,selected_indices]



To-DO: sort out decoder.decoder.n_units = decoder.decoder.filt.C.shape[0]
why cannot we sort out the decoder units

In [16]:
from weights import change_target_kalman_filter_with_a_C_mat

change_target_kalman_filter_with_a_C_mat(decoder.decoder.filt, transformed_C, 
                                         Q = transformed_Q, 
                                         debug = False
                                        )

print(decoder.decoder.n_units)
decoder.decoder.n_features = decoder.decoder.filt.C.shape[0]

print(decoder.decoder.n_features)

20
4


In [17]:
print(decoder.decoder(trans_neural_obs))

[[ 4.00867411]
 [ 0.        ]
 [ 3.6179261 ]
 [13.77098461]
 [ 0.        ]
 [12.43149166]
 [ 1.        ]]


## added the features

In [18]:
num_add_feat = 2
(num_old_feats,num_states) = decoder.decoder.filt.C.shape
num_new_feats = num_add_feat + num_old_feats

#add zero rows to the observation matrix
prev_C = decoder.decoder.filt.C
new_rows = np.zeros((num_add_feat, num_states))
new_C =  np.vstack((prev_C, new_rows))

new_C.shape

#add rows and columns to the observation matrix
prev_Q = decoder.decoder.filt.Q
new_rows = np.zeros((num_add_feat, num_old_feats))
print(prev_Q.shape)
new_Q = np.vstack((prev_Q, new_rows))

new_cols = np.zeros((num_new_feats, num_add_feat))
new_Q = np.hstack((new_Q, new_cols))

# assign back to the 
change_target_kalman_filter_with_a_C_mat(decoder.decoder.filt, new_C, Q = new_Q)


    

(4, 4)
C matrix before
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  4.96162714e-01
   0.00000000e+00  4.83642725e-04  1.04880308e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -4.94739263e-01
   0.00000000e+00 -3.62002311e-03  1.05552288e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.57372307e-02
   0.00000000e+00  5.05239899e-01  1.04127074e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -2.47342267e-02
   0.00000000e+00 -5.01282137e-01  1.03067398e+00]]
C matrix after
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  4.96162714e-01
   0.00000000e+00  4.83642725e-04  1.04880308e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -4.94739263e-01
   0.00000000e+00 -3.62002311e-03  1.05552288e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.57372307e-02
   0.00000000e+00  5.05239899e-01  1.04127074e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -2.47342267e-02
   0.00000000e+00 -5.01282137e-01  1.03067398e+00]
 [ 0.00000000e+00  0.0000

In [19]:
print(new_Q)

[[ 1.63863066  0.57053166  0.01753904  0.03551428  0.          0.        ]
 [ 0.57053166  1.51080346 -0.00306715  0.04064861  0.          0.        ]
 [ 0.01753904 -0.00306715  1.663034    0.64641987  0.          0.        ]
 [ 0.03551428  0.04064861  0.64641987  1.71429543  0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.        ]]


In [20]:
A = np.arange(9).reshape((3,3,))
A

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]])

In [21]:
prev_C.shape

(4, 7)

In [22]:
new_rows = np.zeros((num_new_feat, num_states))
new_C =  np.vstack((prev_C, new_rows))

new_C.shape

NameError: name 'num_new_feat' is not defined

# understand controller

In [ ]:
from riglib.bmi.feedback_controllers import LQRController
Q = np.mat(np.diag([1., 1, 1, 0, 0, 0, 0]))
R = 10000*np.mat(np.diag([1., 1., 1.]))

current_state = decoder.decoder.get_state()
print('current_state')
print(current_state)


print('intended state:')
print(intended_state)

A,B,W = ssm.get_ssm_matrices()
fb_ctrl = LQRController(A, B, Q, R)
ctrl = fb_ctrl.calc_next_state(current_state, intended_state)
print('control input into the state')
print(ctrl)

print('encoder generate control')
print(encoder(ctrl))

# change the learner  as well

In [ ]:
from riglib.bmi.clda import KFSmoothbatch

sb_updater = KFSmoothbatch(batch_time = 20,
                          half_life = 10)